In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import os

### Open file and parse it with bs4

In [2]:
f = open('data_files/reut2-000.sgm', encoding='utf-8', errors='ignore')
dataFile = f.read()
soup = BeautifulSoup(dataFile,'lxml')

FileNotFoundError: [Errno 2] No such file or directory: 'data_files/reut2-000.sgm'

In [ ]:
type(soup)

In [ ]:
f.name

### Check structure of an article

In [ ]:
content = soup.find_all('reuters')

In [ ]:
len(content)

Each file has 1000 articles

In [ ]:
content[5]

### Look at topic list

In [ ]:
topics = np.genfromtxt("data_files/all-topics-strings.lc.txt", 
                      delimiter='\n', dtype=None, encoding=None)
topics

### Find all topics and turn them in to text

In [ ]:
topics = soup.find_all('topics')
topic_list = list()

In [ ]:
for x in topics:
    # turn bs4.tag into text
    words = [i.text for i in x]
    #append text to list
    topic_list.append(words)

### Make a dataframe with the topics

Make function to pull out **'earn'** topic

I only care about the 'earn' topic for now, so I'm making it the only one that matters, but I'm not clear what the ones that have no topic mean. I'm calling those 'blank'.

In [ ]:
def pull_out_earn_topic(topic_list):
    for i, topic in enumerate(topic_list):
        
        # format is a list of strings, so this loop removes topics from nested list
        article_topics = ''
        for word in topic:
            article_topics += (word + ' ')
            
        # assign correct topic 
        if not article_topics:
            topic_list[i] = 'blank'
        elif 'earn' in article_topics:
            topic_list[i] = 'earn'
        else:
            topic_list[i] = 'other'
    
    return topic_list

In [ ]:
test_list = pull_out_earn_topic(topic_list[0:100])
test_list[0:10]

Test case works. Make the whole dataframe.

In [ ]:
topics_for_df = pull_out_earn_topic(topic_list)
df=pd.DataFrame(topics_for_df, columns=['topics'])

In [ ]:
df

### Make list of article content

In [ ]:
all_text = soup.find_all("text")
len(all_text)

In [ ]:
list_all_text = list()
for text in all_text:
    
    # getting just the text from the element
    # stripping out the newline indicator
    working_text = text.get_text().replace("\n", " ")
    
    # removing extra spaces
    working_text = ' '.join(working_text.split())
    
    # appending to list
    list_all_text.append(working_text)

In [ ]:
df['text'] = list_all_text

In [ ]:
df

Now that I got it working with one .sgm file, do the entire dataset.

# Join all datafiles

- make an empty df with the two columns
- loop through each file
    - read the file and make it a bs4 object
    - find the topics
        - make an empty list
        - loop through the topics and turn the bs4 tags into text
        - run the function on the list to make it the topics for the df
    - find the text
        - make an empty list
        - run the loop using `.get_text()`
            - clean the text in the loop
    - add the lists to df using `add_to_df` function
  


### Define functions

In [ ]:
def pull_out_earn_topic(topic_list):
    for i, topic in enumerate(topic_list):
        
        # format is a list of strings, so this loop removes topics from nested list
        article_topics = ''
        for word in topic:
            article_topics += (word + ' ')
            
        # assign correct topic 
        if not article_topics:
            topic_list[i] = 'blank'
        elif 'earn' in article_topics:
            topic_list[i] = 'earn'
        else:
            topic_list[i] = 'other'
    
    return topic_list

In [ ]:
def make_bs4(file):
    filename = os.path.join("data_files", file)
    f = open(filename, 'r', encoding='utf-8', errors='ignore')
    dataFile = f.read()
    print(f.name)
        
    # make it a bs4 object
    soup = BeautifulSoup(dataFile,'lxml')
    return soup

In [ ]:
def add_to_df(topics, texts, df):
    for i, topic in enumerate(topics):
        new_row = pd.Series([topics[i], texts[i]], index=df.columns)
        df = df.append(new_row, ignore_index=True)
    return df

### Make empty dataframe

In [ ]:
df = pd.DataFrame(columns=['topic', 'text'])
df

### Execute big loop to add all the data to the dataframe

In [3]:

for file in os.listdir("data_files/"): 

    if file.endswith(".sgm"):
        
        # for each sgm file, read it and make it a bs4 object
        soup = make_bs4(file)
        
        # isolate topics
        topic_list = list()
        
        topics = soup.find_all('topics')
        
        for x in topics:
            # turn bs4.tag into text
            words = [i.text for i in x]
            #append text to list
            topic_list.append(words)
        
        topic_list = pull_out_earn_topic(topic_list)
        
        # isolate text
        list_all_text = list()
        
        all_text = soup.find_all("text")
        
        for text in all_text:
            
            # getting just the text from the element
            # stripping out the newline indicator
            working_text = text.get_text().replace("\n", " ")
            
            # removing extra spaces
            working_text = ' '.join(working_text.split())
            
            # appending to list
            list_all_text.append(working_text)
        
        # add the article's topic and the article's text to the df         
        df = add_to_df(topic_list, list_all_text, df)
df

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'data_files/'

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23000 entries, 0 to 22999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   topic   23000 non-null  object
 1   text    23000 non-null  object
dtypes: object(2)
memory usage: 359.5+ KB


In [175]:
df[0:10]

,topic,text
0,other,"BAHIA COCOA REVIEW SALVADOR, Feb 26 - Showers ..."
1,blank,STANDARD OIL <SRD> TO FORM FINANCIAL UNIT CLEV...
2,blank,TEXAS COMMERCE BANCSHARES <TCB> FILES PLAN HOU...
3,blank,TALKING POINT/BANKAMERICA <BAC> EQUITY OFFER b...
4,other,NATIONAL AVERAGE PRICES FOR FARMER-OWNED RESER...
5,other,ARGENTINE 1986/87 GRAIN/OILSEED REGISTRATIONS ...
6,blank,"RED LION INNS FILES PLANS OFFERING PORTLAND, O..."
7,blank,"USX <X> DEBT DOWGRADED BY MOODY'S NEW YORK, Fe..."
8,earn,CHAMPION PRODUCTS <CH> APPROVES STOCK SPLIT RO...
9,other,COMPUTER TERMINAL SYSTEMS <CPML> COMPLETES SAL...


In [178]:
df[990:1001]

,topic,text
990,other,U.S. ASKS JAPAN END AGRICULTURE IMPORT CONTROL...
991,blank,U.S. FINANCIAL ANALYSTS - March 3 Wilcox/Gibbs...
992,blank,U.S. DIVIDEND MEETINGS - MARCH 3 Mickelberry C...
993,blank,U.S. SHAREHOLDER MEETINGS - MARCH 3 None Repor...
994,earn,FIRSTCORP <FCR> SEES GAIN ON CONDEMNATION RALE...
995,blank,ASHTON-TATE <TATE> TO OFFER COMMON SHARES TORR...
996,blank,KEYCORP <KEY> REGISTERS SUBORDINATED NOTES ALB...
997,earn,<NATIONAL SEA PRODUCTS LTD> 4TH QTR NET HALIFA...
998,other,U.K. MONEY MARKET SHORTAGE FORECAST REVISED DO...
999,other,NATIONAL AMUSEMENTS AGAIN UPS VIACOM <VIA> BID...


In [18]:
df.iloc[14792]

topic    blank
text          
Name: 14792, dtype: object

### Checking to see if there are any null or blank strings

In [19]:
np.where(pd.isnull(df))

(array([], dtype=int64), array([], dtype=int64))

In [21]:
np.where(df.applymap(lambda x: x == ''))

(array([14792], dtype=int64), array([1], dtype=int64))

In [22]:
df.iloc[14792]

topic    blank
text          
Name: 14792, dtype: object

It seems row 14792 has empty string. Remove it, and double-check.

In [23]:
df = df.drop([14792])
np.where(df.applymap(lambda x: x == ''))

(array([], dtype=int64), array([], dtype=int64))

In [ ]:
Looks OK.

In [24]:
df.to_csv('data_files/topics_and_text.csv', index=False)